In [1]:
from urllib import request
#from fake_useragent import UserAgent, VERSION
from urllib import parse
import json
import requests
import re
import pandas as pd
from openpyxl import Workbook
from fake_useragent import UserAgent
from bs4 import BeautifulSoup as bs
import time
from tqdm import tqdm
ua = UserAgent(path='fake_useragent.json')

In [2]:
closedf = pd.read_feather('BasicFactor_Close.txt').set_index('time')
stockse = closedf.columns

In [3]:
#测试
url = 'https://emweb.securities.eastmoney.com/PC_HSF10/CompanySurvey/PageAjax?code=shT00018'
uastr = ua.random
headers = {'User-Agent':uastr}
res = requests.get(url=url,headers=headers)
res.text

'{"status":-1,"message":"股票代码不合法"}'

In [4]:
#爬取东方财富网上的公司信息
info_all = []
#每次只爬1000股 保存五个csv文件
stocklist = stockse[5000:6000]
for stock in tqdm(stocklist):
    stockcode = stock[-2:] + stock[:6]
    url = 'https://emweb.securities.eastmoney.com/PC_HSF10/CompanySurvey/PageAjax?code={}'.format(stockcode)
    uastr = ua.random
    headers = {'User-Agent':uastr}
    res = requests.get(url=url,headers=headers)
    try:
        info = json.loads(res.text)['jbzl'][0]
        if info == None:
            info_all.append([])
            continue
        infolist = [info['INDEDIRECTORS'],info['ORG_PROFILE'],info['BUSINESS_SCOPE']]
        info_all.append(infolist)
        time.sleep(0.5)
    except:
        info_all.append([])
infodf = pd.DataFrame(info_all)
infodf.columns = ['name','introduction','scope']
infodf.index = stocklist

100%|██████████| 432/432 [04:56<00:00,  1.46it/s]


In [6]:
infodf.to_csv('info5.csv')

In [7]:
df0 = pd.read_csv('info0.csv')
df1 = pd.read_csv('info1.csv')
df2 = pd.read_csv('info2.csv')
df3 = pd.read_csv('info3.csv')
df4 = pd.read_csv('info4.csv')
df5 = pd.read_csv('info5.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'info1.csv'

In [87]:
finaldf = pd.concat([df0,df1,df2,df3,df4,df5])
finaldf.to_csv('infoall.csv')